<a href="https://colab.research.google.com/github/aviral2cover/Credit-Risk-Prediction-with-Machine-Learning/blob/main/Credit_Risk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Credit Risk Modelling using Machine Learning**#

Before we begin our modelling, let me explain how the system works. Banks build a method to match borrowers who are seeking a loan with investors that are willing to lend money in hopes of returns from the interest. Each borrower fills out an application with details about their financial history. The bank then evaluates the borrowers application and assign an interest rate to the borrower. 

Some marketplaces match borrowers that are seeking a loan with investors that are willing to invest to make profits on interests. Banks charge a higher interest rate to the borrowers who seem to be riskier investments. They have their own criteria for deciding the interest rate. However over the years, many fintech companies have adopted **Machine Learning** to do the job. 

We will be building one such model of our own to demonstrate a complete Machine Learning pipeline, including data cleaning, feature selections and choosing algorithms. 

##Objective##

To build a Machine Learning model that can accurately predict if a borrower will pay off their loan on time. 

##Dataset##

The dataset that we're going to use is from a marketplace called 'Lending Club'. They release data on all their approved and declined loan aplications. The dataset contains information about the approved loans, 

##Assumptions##

We need to keep in mind that in a situation where commodity or money is involved, it is usually prudent to have a conservative approach to building Machine Learning algorithms, in order to minimizing risk of incurring losses. The other strategy is to make several investments ranging from low to high risk returns. 

I have designed this model from the perspecrive of a conservative investor willing to invest in safe options. 


###Data Cleaning###

Let's start the project by first exploring the data and then cleaning it.

Let's start with reading the file in a dataframe and dropping the duplicates. 




In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Reading the Dataset
path1 = "/content/drive/MyDrive/Credit Risk/loans_2007.csv"
path2 = "/content/drive/MyDrive/Credit Risk/filtered_loans_2007.csv"
path3 = "/content/drive/MyDrive/Credit Risk/clean_loans_2007.csv"
import pandas as pd
loans_2007 = pd.read_csv(path1)
loans_2007.drop_duplicates()


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,1077501,1296599.0,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,Dec-2011,Fully Paid,n,credit_card,Computer,860xx,AZ,27.65,0.0,Jan-1985,1.0,3.0,0.0,13648.0,83.7%,9.0,f,0.00,0.00,5863.155187,5833.84,5000.00,863.16,0.00,0.00,0.00,Jan-2015,171.62,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
1,1077430,1314167.0,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,Dec-2011,Charged Off,n,car,bike,309xx,GA,1.00,0.0,Apr-1999,5.0,3.0,0.0,1687.0,9.4%,4.0,f,0.00,0.00,1008.710000,1008.71,456.46,435.17,0.00,117.08,1.11,Apr-2013,119.66,Sep-2013,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
2,1077175,1313524.0,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,NaN,10+ years,RENT,12252.0,Not Verified,Dec-2011,Fully Paid,n,small_business,real estate business,606xx,IL,8.72,0.0,Nov-2001,2.0,2.0,0.0,2956.0,98.5%,10.0,f,0.00,0.00,3005.666844,3005.67,2400.00,605.67,0.00,0.00,0.00,Jun-2014,649.91,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
3,1076863,1277178.0,10000.0,10000.0,10000.0,36 months,13.49%,339.31,C,C1,AIR RESOURCES BOARD,10+ years,RENT,49200.0,Source Verified,Dec-2011,Fully Paid,n,other,personel,917xx,CA,20.00,0.0,Feb-1996,1.0,10.0,0.0,5598.0,21%,37.0,f,0.00,0.00,12231.890000,12231.89,10000.00,2214.92,16.97,0.00,0.00,Jan-2015,357.48,Apr-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
4,1075358,1311748.0,3000.0,3000.0,3000.0,60 months,12.69%,67.79,B,B5,University Medical Group,1 year,RENT,80000.0,Source Verified,Dec-2011,Current,n,other,Personal,972xx,OR,17.94,0.0,Jan-1996,0.0,15.0,0.0,27783.0,53.9%,38.0,f,461.73,461.73,3581.120000,3581.12,2538.27,1042.85,0.00,0.00,0.00,Jun-2016,67.79,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42533,72176,70868.0,2525.0,2525.0,225.0,36 months,9.33%,80.69,B,B3,NaN,< 1 year,RENT,110000.0,Not Verified,Jun-2007,Does not meet the credit policy. Status:Fully ...,n,other,Car repair bill,100xx,NY,10.00,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,f,0.00,0.00,2904.498829,258.82,2525.00,379.50,0.00,0.00,0.00,Jun-2010,82.03,May-2007,NaN,1.0,INDIVIDUAL,NaN,NaN,NaN,NaN,NaN
42534,71623,70735.0,6500.0,6500.0,0.0,36 months,8.38%,204.84,A,A5,NaN,< 1 year,NONE,NaN,Not Verified,Jun-2007,Does not meet the credit policy. Status:Fully ...,n,other,Buying a car,100xx,NY,4.00,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,f,0.00,0.00,7373.904962,0.00,6500.00,873.90,0.00,0.00,0.00,Jun-2010,205.32,Aug-2007,NaN,1.0,INDIVIDUAL,NaN,NaN,NaN,NaN,NaN
42535,70686,70681.0,5000.0,5000.0,0.0,36 months,7.75%,156.11,A,A3,Homemaker,10+ years,MORTGAGE,70000.0,Not Verified,Jun-2007,Does not meet the credit policy. Status:Fully ...,n,other,Aroundthehouse,068xx,CT,8.81,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,f,0.00,0.00,5619.762090,0.00,5000.00,619.76,0.00,0.00,0.00,Jun-2010,156.39,Feb-2015,NaN,1.0,INDIVIDUAL,NaN,NaN,NaN,NaN,NaN
42536,Total amount funded in policy code 1: 471701350,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


To familiarize ourselves with the columns, let's print the first row and display the number of columns.

In [ ]:
# First row 
print(loans_2007.iloc[0])
print(loans_2007.shape[1])

id                                1077501
member_id                      1.2966e+06
loan_amnt                            5000
funded_amnt                          5000
funded_amnt_inv                      4975
term                            36 months
int_rate                           10.65%
installment                        162.87
grade                                   B
sub_grade                              B2
emp_title                             NaN
emp_length                      10+ years
home_ownership                       RENT
annual_inc                          24000
verification_status              Verified
issue_d                          Dec-2011
loan_status                    Fully Paid
pymnt_plan                              n
purpose                       credit_card
title                            Computer
zip_code                            860xx
addr_state                             AZ
dti                                 27.65
delinq_2yrs                       

There are a total of **52** columns and it looks like we have a lot of cleaning to do. For example, to read **'emp_length'** as an int or a float, we have to turn it into a compatible format. 

While data cleaning, we need to also take into account the columns that can alter our results: 

1. Some columns represent data that convey a future occurence. For example, 'funded-amount' column represents the amount paid off by the borrower. This information is not available to the investor. 

2. Columns that are formatted poorly that may lead to an error in code or inaccurate results.

3. Columns that have no relationship with the borrower's ability to pay off the loan (Eg: ID)

4. Containt redundant information.

5. Have a lot of missing values.

In the next code block, I have removed some columns, and formatted data that I felt were appropriate for the sake of running this model.



In [ ]:
# Dropping columns that fit the description of at least one of the conditions stated above.

loans_2007 = loans_2007.drop(["id", "member_id", "pymnt_plan" ,"funded_amnt", "funded_amnt_inv", "grade", "sub_grade", "emp_title", "issue_d", "zip_code", "out_prncp", "out_prncp_inv", "total_pymnt", "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee", "recoveries", "collection_recovery_fee", "last_pymnt_d", "last_pymnt_amnt"], axis=1)

The **loan_status** column conveys whether the borrower could finally pay back the borrowed amount. Hence this should be considered the target column, i.e the label column. Let's explore its values.

In [ ]:
print(loans_2007['loan_status'].value_counts())

Fully Paid                                             33136
Charged Off                                             5634
Does not meet the credit policy. Status:Fully Paid      1988
Current                                                  961
Does not meet the credit policy. Status:Charged Off      761
Late (31-120 days)                                        24
In Grace Period                                           20
Late (16-30 days)                                          8
Default                                                    3
Name: loan_status, dtype: int64


The investors are concerned only with whether or not a borrower would pay back the loan or not. Hence, we would remove all the other rows except the ones that contain **Fully Paid** and **Charged Off**. Also not necessary, I have labeled **Fully Paid** as 1, and **Charged Off** as 0. The model can now be treated as a binary classification problem. 

In [ ]:
loans_2007 = loans_2007[(loans_2007['loan_status'] == "Fully Paid") | (loans_2007['loan_status'] == "Charged Off")| (loans_2007['loan_status'] == "Late (31-120 days)")| (loans_2007['loan_status'] == "In Grace Period")| (loans_2007['loan_status'] == "Default")| (loans_2007['loan_status'] == "Late (16-30 days)")]

status_replace = {
    "loan_status" : {
        "Fully Paid": 1,
        "Charged Off": 0,
        "Late (31-120 days)":0.5,
        "In Grace Period": 0.8,                                       
        "Default":0,
        "Late (16-30 days)": 0.4                                       
                                           
 
    }
}

loans_2007 = loans_2007.replace(status_replace)

Let's remove columns that have only one unique value, since they add no information to the model

In [ ]:
orig_columns = loans_2007.columns
drop_columns = []
for col in orig_columns:
    col_series = loans_2007[col].dropna().unique()
    if len(col_series) == 1:
        drop_columns.append(col)
loans_2007 = loans_2007.drop(drop_columns, axis=1)
print(drop_columns)

['initial_list_status', 'collections_12_mths_ex_med', 'policy_code', 'application_type', 'acc_now_delinq', 'chargeoff_within_12_mths', 'delinq_amnt', 'tax_liens']


Let's investigate the null values now. A Machine Learning model will throw an error if it comes across a Null value. 

In [ ]:
loans = loans_2007
null_counts = loans.isnull().sum()
print(null_counts[null_counts>0])

emp_length              1040
title                     11
revol_util                50
last_credit_pull_d         2
pub_rec_bankruptcies     697
dtype: int64


In [ ]:
print(loans.pub_rec_bankruptcies.value_counts(normalize=True, dropna=False))

0.0    0.939446
1.0    0.042473
NaN    0.017952
2.0    0.000129
Name: pub_rec_bankruptcies, dtype: float64


We see that **pub_rec_bankruptcies** has very little variablity. Hence we drop it. Also, we drop all the other rows with Null values.

In [ ]:
loans = loans.drop("pub_rec_bankruptcies", axis=1)
loans = loans.dropna(axis=0)
print(loans.dtypes.value_counts())

object     11
float64    11
dtype: int64


Machine Learning models operate on float and int values. Therefore, we need to deal with columns with **object** datatypes. We do this by converting them into dummy variables. TO do that, I will have to first clean them to make them compatible.

In [ ]:
object_columns_df = loans.select_dtypes(include=["object"])
print(object_columns_df.iloc[0])

term                     36 months
int_rate                    10.65%
emp_length               10+ years
home_ownership                RENT
verification_status       Verified
purpose                credit_card
title                     Computer
addr_state                      AZ
earliest_cr_line          Jan-1985
revol_util                   83.7%
last_credit_pull_d        Jun-2016
Name: 0, dtype: object


Some look like they can be converted into categorical values. Others look more apt for ordinal values. In the next block, I have displayed values of columns that look like they are categorical.

In [ ]:
cols = ['home_ownership', 'verification_status', 'emp_length', 'term', 'addr_state']
for c in cols:
    print(loans[c].value_counts())

RENT        18133
MORTGAGE    16714
OWN          2780
OTHER          96
NONE            3
Name: home_ownership, dtype: int64
Not Verified       16286
Verified           11884
Source Verified     9556
Name: verification_status, dtype: int64
10+ years    8561
< 1 year     4518
2 years      4308
3 years      4026
4 years      3359
5 years      3206
1 year       3181
6 years      2177
7 years      1715
8 years      1446
9 years      1229
Name: emp_length, dtype: int64
 36 months    28234
 60 months     9492
Name: term, dtype: int64
CA    6785
NY    3617
FL    2710
TX    2619
NJ    1778
IL    1448
PA    1444
VA    1348
GA    1324
MA    1275
OH    1152
MD    1011
AZ     808
WA     788
CO     748
NC     729
CT     712
MI     679
MO     648
MN     583
NV     467
SC     454
WI     428
OR     422
LA     420
AL     420
KY     312
OK     285
KS     250
UT     249
AR     229
DC     209
RI     194
NM     181
WV     164
HI     162
NH     157
DE     110
AK      77
MT      77
WY      76
SD      60
VT  

In [ ]:
print(loans["title"].value_counts())
print(loans["purpose"].value_counts())

Debt Consolidation                  2071
Debt Consolidation Loan             1604
Personal Loan                        625
Consolidation                        489
debt consolidation                   468
                                    ... 
buissness                              1
Fixing up rental to reduce debt        1
kerrys loan                            1
Consolidate credit card payments       1
new transmission                       1
Name: title, Length: 18896, dtype: int64
debt_consolidation    17779
credit_card            4916
other                  3716
home_improvement       2813
major_purchase         2084
small_business         1723
car                    1459
wedding                 916
medical                 656
moving                  552
house                   357
vacation                349
educational             312
renewable_energy         94
Name: purpose, dtype: int64


In [ ]:
mapping_dict = {
    "emp_length": {
        "10+ years": 10,
        "9 years": 9,
        "8 years": 8,
        "7 years": 7,
        "6 years": 6,
        "5 years": 5,
        "4 years": 4,
        "3 years": 3,
        "2 years": 2,
        "1 year": 1,
        "< 1 year": 0,
        "n/a": 0
    }
}
loans = loans.drop(["last_credit_pull_d", "earliest_cr_line", "addr_state", "title"], axis=1)
loans["int_rate"] = loans["int_rate"].str.rstrip("%").astype("float")
loans["revol_util"] = loans["revol_util"].str.rstrip("%").astype("float")
loans = loans.replace(mapping_dict)

In the code above, I have cleaned the relevant columns, dropped a few more columns, and converted the values of **emp_length** into numerical values.

In the code below, I have encoded all the discrete categorical variables as dummy variables.

In [ ]:
cat_columns = ["home_ownership", "verification_status", "purpose", "term"]
dummy_df = pd.get_dummies(loans[cat_columns])
loans = pd.concat([loans, dummy_df], axis=1)
loans = loans.drop(cat_columns, axis=1)

In [ ]:
loans

,loan_amnt,int_rate,installment,emp_length,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,home_ownership_MORTGAGE,home_ownership_NONE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,purpose_car,purpose_credit_card,purpose_debt_consolidation,purpose_educational,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,term_ 36 months,term_ 60 months
0,5000.0,10.65,162.87,10,24000.0,1.0,27.65,0.0,1.0,3.0,0.0,13648.0,83.7,9.0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,2500.0,15.27,59.83,0,30000.0,0.0,1.00,0.0,5.0,3.0,0.0,1687.0,9.4,4.0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,2400.0,15.96,84.33,10,12252.0,1.0,8.72,0.0,2.0,2.0,0.0,2956.0,98.5,10.0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
3,10000.0,13.49,339.31,10,49200.0,1.0,20.00,0.0,1.0,10.0,0.0,5598.0,21.0,37.0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
5,5000.0,7.90,156.46,3,36000.0,1.0,11.20,0.0,3.0,9.0,0.0,7963.0,28.3,12.0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39781,2500.0,8.07,78.42,4,110000.0,1.0,11.33,0.0,0.0,13.0,0.0,7274.0,13.1,40.0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0
39782,8500.0,10.28,275.38,3,18000.0,1.0,6.40,1.0,1.0,6.0,0.0,8847.0,26.9,9.0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0
39783,5000.0,8.07,156.84,0,100000.0,1.0,2.30,0.0,0.0,11.0,0.0,9698.0,19.4,20.0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0
39784,5000.0,7.43,155.38,0,200000.0,1.0,3.72,0.0,0.0,17.0,0.0,85607.0,0.7,26.0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0


##Modelling##

Finally, we have arrived at a point where our dataset is ready to be trained.

Before we dive in, let's describe some of the metrics I'll be using to determine the performance of the model.

**False Positive**- A situation where the model predicts that the loan will be paid off on time, but it actually doesn't.

**False Negative**- A situation where the model predicts that the loan will not be paid off on time, but it actually does.

**True Positive**- Model correctly predicts that the loan will be paid off on time.

**True Negative**- Model correctly predicts that the loan will not be paid off on time.


**False Positive Rate = False Positive / (False Positive + True Negative)**

**True Positive Rate = True Positive / (True Positive + False Negative) (Precision)**

Now let's test these equations by plugging in 1s for each row in the negative column



In [ ]:
import pandas as pd
import numpy

# Predict that all loans will be paid off on time.
predictions = pd.Series(numpy.ones(loans.shape[0]))
# False positives.
fp_filter = (predictions == 1) & (loans["loan_status"] == 0)
fp = len(predictions[fp_filter])

# True positives.
tp_filter = (predictions == 1) & (loans["loan_status"] == 1)
tp = len(predictions[tp_filter])

# False negatives.
fn_filter = (predictions == 0) & (loans["loan_status"] == 1)
fn = len(predictions[fn_filter])

# True negatives
tn_filter = (predictions == 0) & (loans["loan_status"] == 0)
tn = len(predictions[tn_filter])

# Rates
tpr = tp / (tp + fn)
fpr = fp / (fp + tn)

print(tpr)
print(fpr) # Precision 

1.0
1.0


Both **fpr** and **tpr** were 1. This is because I predicted that loans will be paid off by all the borrowers. This means that I correctly identified all of the good loans (true positive rate), but I also incorrectly identified all of the bad loans (false positive rate).

Now we can move on to building a Machine Learning model.

For binary classification, a good first step is to use Logistic Regression. I have used the scikit-learn library to speed up the process, instead of computing the sigmoid functions. 

In order to build a test set, we will be using **k-fold cross validation** algorithm.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn import utils
lab_enc = preprocessing.LabelEncoder()
target = loans["loan_status"]
encoded = lab_enc.fit_transform(target)
lr = LogisticRegression(max_iter=1000)
cols = loans.columns
train_cols = cols.drop("loan_status")
features = loans[train_cols]
lr.fit(features, encoded)
predictions = lr.predict(features)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [ ]:
pd.Series(predictions).value_counts()

4    37654
0       72
dtype: int64

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict
lr = LogisticRegression(max_iter=1000)
predictions = cross_val_predict(lr, features, target, cv=3)
predictions = pd.Series(predictions)
# False positives.
fp_filter = (predictions == 1) & (loans["loan_status"] == 0)
fp = len(predictions[fp_filter])

# True positives.
tp_filter = (predictions == 1) & (loans["loan_status"] == 1)
tp = len(predictions[tp_filter])

# False negatives.
fn_filter = (predictions == 0) & (loans["loan_status"] == 1)
fn = len(predictions[fn_filter])

# True negatives
tn_filter = (predictions == 0) & (loans["loan_status"] == 0)
tn = len(predictions[tn_filter])
# Rates
tpr = tp  / (tp + fn)
fpr = fp  / (fp + tn)
print(tpr)
print(fpr)

0.9980007865757735
0.9990128331688055


While examining our dataset, we could notice that there is a heavy imbalance between the number of **Fully Paid** and **Charged Off** values. As a result, the model will tend to displau a high true positive rate, but also a very hight false positive rate. 

To offset this bias, we can assign a parameter **class_weight** and set it to "balanced". This makes scikit-learn penalize a misclassification of the minority class in a stricter way. 

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict
lr = LogisticRegression(class_weight="balanced", max_iter=200)
predictions = cross_val_predict(lr, features, target, cv=3)
predictions = pd.Series(predictions)

# False positives.
fp_filter = (predictions == 1) & (loans["loan_status"] == 0)
fp = len(predictions[fp_filter])

# True positives.
tp_filter = (predictions == 1) & (loans["loan_status"] == 1)
tp = len(predictions[tp_filter])

# False negatives.
fn_filter = (predictions == 0) & (loans["loan_status"] == 1)
fn = len(predictions[fn_filter])

# True negatives
tn_filter = (predictions == 0) & (loans["loan_status"] == 0)
tn = len(predictions[tn_filter])

# Rates
tpr = tp / (tp + fn)
fpr = fp / (fp + tn)

print(tpr)
print(fpr)

0.5120608285264814
0.5032576505429418


This significantly improved the false positive rate, at the expense of lowering the true positive rate. As I mentioned earlier, I am attempting to design the model from a conservative investor's standpoint, who would avoid taking the risk of losing money on bad loans. 

The current model is penalizing the msclassification of the minority labels by a factor of 5.89. Let's try to increase this factor to 10 in an attempt to improve the false positive rate.



In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict
penalty = {
   0: 7,
    4:1
    
}

lr = LogisticRegression(class_weight=penalty, max_iter=200)
predictions = cross_val_predict(lr, features, encoded, cv=3)
predictions = pd.Series(predictions)

# False positives.
fp_filter = (predictions == 4) & (loans["loan_status"] == 0)
fp = len(predictions[fp_filter])

# True positives.
tp_filter = (predictions == 4) & (loans["loan_status"] == 1)
tp = len(predictions[tp_filter])

# False negatives.
fn_filter = (predictions == 0) & (loans["loan_status"] == 1)
fn = len(predictions[fn_filter])

# True negatives
tn_filter = (predictions == 0) & (loans["loan_status"] == 0)
tn = len(predictions[tn_filter])

# Rates
tpr = tp / (tp + fn)
fpr = fp / (fp + tn)

print(tpr)
print(fpr)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


0.4027164130256914
0.382538431217974


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [ ]:
print(tpr)
print(fpr)

0.4027164130256914
0.382538431217974


This is a massie improvement from my original model that incorrectly predicted almost all the bad loans. The trained model has a false positive rate of only 15%, reducing it from 99% in the original model.

Let's try another classification algorithm. This time a Random Forest Classifier. Random forests are more appropriate with nonlinear data. Logistic regressions on the other hand are only able to work with linear data. My assumption is that training a random forest may give us more accuracy due to columns that correlate nonlinearly with our target column.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict

penalty = {
    0: 7,
    4: 1
}

rf = RandomForestClassifier(class_weight=penalty, random_state=1)
predictions = cross_val_predict(rf, features, encoded, cv=3)
predictions = pd.Series(predictions)

# False positives.
fp_filter = (predictions == 4) & (loans["loan_status"] == 0)
fp = len(predictions[fp_filter])

# True positives.`
tp_filter = (predictions == 4) & (loans["loan_status"] == 1)
tp = len(predictions[tp_filter])

# False negatives.
fn_filter = (predictions == 0) & (loans["loan_status"] == 1)
fn = len(predictions[fn_filter])

# True negatives
tn_filter = (predictions == 0) & (loans["loan_status"] == 0)
tn = len(predictions[tn_filter])

# Rates
tpr = tp / (tp + fn)
fpr = fp / (fp + tn)

print(tpr)
print(fpr)

Looks like Random Forest is still heavily weighting on the label 1. 

So our best option is to use Logistic Regression with a penalty factor of 10 on the label 0. 

A conservative investor would make profits as long as the interest rate is high enough to offset the losses from the 15% of borrowers defaulting, and that the pool of the 15% of those that fully pay the loans is large enough to make enough interest money to offset the losses.